In [ ]:
import numpy as np
import pandas as pd
from random import random
import math

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
from GDR import GradientDR
from DensityDimReduction.distance_metric import get_nearest_neighbors
from DensityDimReduction.experiment_utils.get_data import get_dataset
import scipy
import networkit
import sklearn
from sklearn.cluster import OPTICS

In [ ]:
def set_seed(i):
    np.random.seed(i)

Based on code provided by: http://extremelearning.com.au/how-to-generate-uniformly-random-points-on-n-spheres-and-n-balls/

In [ ]:
dims = [2,5,7,10,50,100]
#dims = [2,3]
seeds = [0,1,2,3,4]
#seeds = [0,1]

Make Data

In [ ]:
import datagen

for dim in dims:
    for seed in seeds:
        print(str(dim) + " " + str(seed))
        datagen.run(["", seed, 1000, dim, 5, False, 0, 10, 25])

In [ ]:
directory = os.path.join('data', 'blob')

for dim in dims:
    for seed in seeds:
        set_seed(seed)
        blobs = sklearn.datasets.make_blobs(1000, n_features=dim, centers = 10, center_box= (0,20))

        #print(blobs[0].shape)
        #print(blobs[1])



        blobdata = np.zeros((blobs[0].shape[0],blobs[0].shape[1]+1))
        blobdata[:,:-1] = blobs[0]
        blobdata[:,-1] = blobs[1]

        #print(blobdata)

        dataid = directory + "\\blob_data_1000_10_" + str(dim) + "_" + str(seed)
        np.save(dataid,blobdata)

In [ ]:
directory = os.path.join('data', 'uniform')

for dim in dims:
    for seed in seeds:
        set_seed(seed)
        uniform = np.random.uniform(low=0, high=100, size=[1000,dim])

        #print(uniform)



        uniformdata = np.zeros((uniform.shape[0],uniform.shape[1]+1))
        uniformdata[:,:-1] = uniform

        #print(blobdata)

        dataid = directory + "\\uniform_data_1000_10_" + str(dim) + "_" + str(seed)
        np.save(dataid,uniformdata)

Set Datatsets

In [ ]:
directory = os.path.join('data', 'synth')
datasets =  []
for dim in dims:
    for seed in seeds:
        datasets.append('synth_data_1000_5_' + str(dim) + '_' + str(seed))



points_list = []
labels_list = []
for i in range(len(datasets)):
    file_path = os.path.join(directory, str(datasets[i]) + ".npy")
    synthdata = np.load(file_path)
    points_list.append(synthdata[:,:-1])
    labels_list.append(synthdata[:,-1])

In [ ]:
directory = os.path.join('data', 'blob')
datasets =  []
for dim in dims:
    for seed in seeds:
        datasets.append('blob_data_1000_10_' + str(dim) + '_' + str(seed))



points_list = []
labels_list = []
for i in range(len(datasets)):
    file_path = os.path.join(directory, str(datasets[i]) + ".npy")
    blobdata = np.load(file_path)
    points_list.append(blobdata[:,:-1])
    labels_list.append(blobdata[:,-1])

In [ ]:
directory = os.path.join('data', 'uniform')
datasets =  []
for dim in dims:
    for seed in seeds:
        datasets.append('uniform_data_1000_10_' + str(dim) + '_' + str(seed))



points_list = []
labels_list = []
for i in range(len(datasets)):
    file_path = os.path.join(directory, str(datasets[i]) + ".npy")
    uniformdata = np.load(file_path)
    points_list.append(uniformdata[:,:-1])
    labels_list.append(uniformdata[:,-1])

Show 2-dim Data

In [ ]:
color = plt.cm.tab20(np.linspace(0, 1, 12))
#print(color[x3[:,2].astype('int32')])

plt.figure(figsize=(15,15))
plt.scatter(points_list[0][:,0], points_list[0][:,1], c=color[labels_list[0].astype('int32')], alpha=0.4)
#plt.scatter(centers[:,0], centers[:,1], c='black', alpha=1)

plt.ylim(np.min(points_list[0][:,1]-1), np.max(points_list[0][:,1]+1))
plt.xlim(np.min(points_list[0][:,0]-1), np.max(points_list[0][:,0]+1))

In [ ]:
print(directory)
print(datasets)

In [ ]:
eucliddists = []
manhatdists = []
cosinedists = []
widestdists = []

dsnedists = []
dsnedists2 = []
dsnedists5 = []
dsnedists10 = []

Calculate distances

In [ ]:
eucliddist = 0
manhatdist = 0
cosinedist = 0
dsnedist = 0
dsne2dist = 0
dsne5dist = 0
dsne10dist = 0

print(directory)
print(datasets)

for i in range(len(datasets)):
    
    print(datasets[i])
    
    eucliddist = np.reshape(scipy.spatial.distance.cdist(points_list[i], points_list[i], metric='euclidean'), -1)
    eucfile = directory + "//" + str(datasets[i]) + "_euc"
    np.save(eucfile,eucliddist)
    

    
    manhatdist = np.reshape(scipy.spatial.distance.cdist(points_list[i], points_list[i], metric='cityblock'), -1) 
    manfile = directory + "//" + str(datasets[i]) + "_man"
    np.save(manfile,manhatdist)
    
    cosinedist = np.reshape(scipy.spatial.distance.cdist(points_list[i], points_list[i], metric='cosine'), -1)
    cosfile = directory + "//" + str(datasets[i]) + "_cos"
    np.save(cosfile,cosinedist)
    
    dsnenns = get_nearest_neighbors(points_list[i], 15, min_points=1)
    dsnedist = np.reshape(dsnenns['_all_dists'], -1)
    dsnefile = directory + "//" + str(datasets[i]) + "_dsne"
    np.save(dsnefile,dsnedist)
    
    dsnenns2 = get_nearest_neighbors(points_list[i], 15, min_points=2)
    dsne2dist = np.reshape(dsnenns2['_all_dists'], -1)
    dsne2file = directory + "//" + str(datasets[i]) + "_dsne2"
    np.save(dsne2file,dsne2dist)
    
    dsnenns5 = get_nearest_neighbors(points_list[i], 15, min_points=5)
    dsne5dist = np.reshape(dsnenns5['_all_dists'], -1)
    dsne5file = directory + "//" + str(datasets[i]) + "_dsne5"
    np.save(dsne5file,dsne5dist)
    
    dsnenns10 = get_nearest_neighbors(points_list[i], 15, min_points=10)
    dsne10dist = np.reshape(dsnenns10['_all_dists'], -1)
    dsne10file = directory + "//" + str(datasets[i]) + "_dsne10"
    np.save(dsne10file,dsne10dist)

Load Data

In [ ]:
eucliddists = []
manhatdists = []
cosinedists = []
widestdists = []

dsnedists = []
dsnedists2 = []
dsnedists5 = []
dsnedists10 = []


for i in range(len(datasets)):
    
    print(datasets[i])

    file_path_euc = os.path.join(directory, str(datasets[i]) + "_euc.npy")
    eucliddist = np.load(file_path_euc)
    
    file_path_man = os.path.join(directory, str(datasets[i]) + "_man.npy")
    manhatdist = np.load(file_path_man)
    
    file_path_cos = os.path.join(directory, str(datasets[i]) + "_cos.npy")
    cosinedist = np.load(file_path_cos)
    
    file_path_dsne = os.path.join(directory, str(datasets[i]) + "_dsne.npy")
    dsnedist = np.load(file_path_dsne)
    
    file_path_dsne2 = os.path.join(directory, str(datasets[i]) + "_dsne2.npy")
    dsne2dist = np.load(file_path_dsne2)
    
    file_path_dsne5 = os.path.join(directory, str(datasets[i]) + "_dsne5.npy")
    dsne5dist = np.load(file_path_dsne5)
    
    file_path_dsne10 = os.path.join(directory, str(datasets[i]) + "_dsne10.npy")
    dsne10dist = np.load(file_path_dsne10)
    
    eucliddists.append(eucliddist)
    manhatdists.append(manhatdist)
    cosinedists.append(cosinedist)
    dsnedists.append(dsnedist)
    dsnedists2.append(dsne2dist)
    dsnedists5.append(dsne5dist)
    dsnedists10.append(dsne10dist)

In [ ]:
def gather_values(dists, labels, bins = 30):
    #print(len(dists))
    #print(len(labels))

    label_agreement = np.expand_dims(labels, 0) == np.expand_dims(labels, 1)
    label_agreement = np.reshape(label_agreement, -1)
    is_noise = np.logical_or(np.expand_dims(labels, 0) == -1, np.expand_dims(labels, 1) == -1)
    is_noise = np.reshape(is_noise, -1)
    is_self = np.identity(len(labels))
    is_self = np.reshape(is_self, -1)
    intra_class = dists[np.logical_and(np.logical_and(label_agreement == 1,is_noise==0), is_self == 0)]
    intra_noise_class = dists[np.logical_and(np.logical_and(label_agreement == 1,is_noise==1), is_self == 0)]
    inter_class = dists[np.logical_and(np.logical_and(label_agreement == 0,is_noise==0), is_self == 0)]
    inter_noise_class = dists[np.logical_and(np.logical_and(label_agreement == 0,is_noise==1), is_self == 0)]
    
    notself =  dists[is_self == 0]
    
    mean = np.mean(notself)
    stdev = np.std(notself)
    
    max_value = max(dists)
    min_value = min(notself)
        
    print("min value: " + str(min_value))
    print("max value: " + str(max_value))
    
    
    
    ratio = min_value/max_value
    #print(len(intra_class))
    #print(len(inter_class))
    #print(len(intra_noise_class))
    #print(len(inter_noise_class))
    
    intra_hist = np.histogram(intra_class, bins= bins, range = (0,max_value))
    inter_hist = np.histogram(inter_class, bins= bins, range = (0,max_value))
    intra_noise_hist = np.histogram(intra_noise_class, bins= bins, range = (0,max_value))
    inter_noise_hist = np.histogram(inter_noise_class, bins= bins, range = (0,max_value))

    return intra_hist, inter_hist, intra_noise_hist, inter_noise_hist, ratio, mean, stdev

In [ ]:
def make_graph(points_list, labels_list, num_bin, eucliddist, manhatdist, cosinedist, dsnedist, dsnedist2, dsnedist5, dsnedist10, dimlist, seedlist, datasets): #, dsnedist2):

    title = ""
    if('blob' in datasets[0]):
        title = "Blobs"
    elif('synth' in datasets[0]):
        title = "Density-based" 
    elif('uniform' in datasets[0]):
        title = "Uniform"
    
    colors = plt.cm.tab20(np.linspace(0, 1, 20))
    
    colors = ["deepskyblue", "lightskyblue","deepskyblue", "lightskyblue","deepskyblue", "lightskyblue","deepskyblue", "lightskyblue", "orange", "moccasin" , "green", "lightgreen", "purple", "mediumorchid"]

    ratios_man = []
    ratios_euc = []
    ratios_cos = []
    ratios_dsne = []
    ratios_dsne2 = []
    ratios_dsne5 = []
    ratios_dsne10 = []
    
    means_man = []
    means_euc = []
    means_cos = []
    means_dsne = []
    means_dsne2 = []
    means_dsne5 = []
    means_dsne10 = []
    
    stdevs_man = []
    stdevs_euc = []
    stdevs_cos = []
    stdevs_dsne = []
    stdevs_dsne2 = []
    stdevs_dsne5 = []
    stdevs_dsne10 = []
    
    print(seedlist)
    

    for i in range(len(dimlist)):
        
        
        intra_hists_dsne  = []
        inter_hists_dsne = []
        intra_hists_dsne2 = []
        inter_hists_dsne2 = [] 
        intra_hists_dsne5 = []
        inter_hists_dsne5 = []
        intra_hists_dsne10 = []
        inter_hists_dsne10 = []
        
        intra_hists_euc = []
        inter_hists_euc = []
        intra_hists_man = []
        inter_hists_man = []
        intra_hists_cos = []
        inter_hists_cos = []
        
        k = i * len(seedlist)
        for s in range(len(seedlist)):
            print("Dimensionality: " + str(dimlist[i]) + " Seed: " + str(seedlist[s]) + " Position: " + str(k+s))
            
            #print("DSNE")
            intra_hist_dsne, inter_hist_dsne, _, _, ratio_dsne, mean_dsne, stdev_dsne = gather_values(dsnedist[k+s], labels_list[k+s], num_bin)
            #print("Ratio: " + str(ratio_dsne) + "\n")
            #print("DSNE minPoints 2")
            intra_hist_dsne2, inter_hist_dsne2, _, _, ratio_dsne2, mean_dsne2, stdev_dsne2 = gather_values(dsnedist2[k+s], labels_list[k+s], num_bin)
            #print("Ratio: " + str(ratio_dsne2) + "\n")
            #print("DSNE minPoints 5")
            intra_hist_dsne5, inter_hist_dsne5, _, _, ratio_dsne5, mean_dsne5, stdev_dsne5 = gather_values(dsnedist5[k+s], labels_list[k+s], num_bin)
            #print("Ratio: " + str(ratio_dsne5) + "\n")
            #print("DSNE minPoints 10")
            intra_hist_dsne10, inter_hist_dsne10, _, _, ratio_dsne10, mean_dsne10, stdev_dsne10 = gather_values(dsnedist10[k+s], labels_list[k+s], num_bin)
            #print("Ratio: " + str(ratio_dsne10) + "\n")
            #print("Euclidean")
            intra_hist_euc, inter_hist_euc, _, _, ratio_euc, mean_euc, stdev_euc = gather_values(eucliddist[k+s], labels_list[k+s], num_bin)
            #print("Ratio: " + str(ratio_euc) + "\n")
            #print("Manhattan")
            intra_hist_man, inter_hist_man, _, _, ratio_man, mean_man, stdev_man = gather_values(manhatdist[k+s], labels_list[k+s], num_bin)
            #print("Ratio: " + str(ratio_man) + "\n")
            #print("Cosine")
            intra_hist_cos, inter_hist_cos, _, _, ratio_cos, mean_cos, stdev_cos = gather_values(cosinedist[k+s], labels_list[k+s], num_bin)
            #print("Ratio: " + str(ratio_cos))
            
            
            #print("Intra sum: " + str(np.sum(intra_hist_dsne[0])))
            #print("Inter sum: " + str(np.sum(inter_hist_dsne[0])))
            #print("Full sum: " + str(np.sum(intra_hist_dsne[0]) + np.sum(inter_hist_dsne[0]) ))
            
            
            #print("---")
            
            
            intra_hists_dsne.append(intra_hist_dsne[0])
            
            
            inter_hists_dsne.append(inter_hist_dsne[0])
            intra_hists_dsne2.append(intra_hist_dsne2[0])
            inter_hists_dsne2.append(inter_hist_dsne2[0])
            intra_hists_dsne5.append(intra_hist_dsne5[0])
            inter_hists_dsne5.append(inter_hist_dsne5[0])
            intra_hists_dsne10.append(intra_hist_dsne10[0])
            inter_hists_dsne10.append(inter_hist_dsne10[0])

            intra_hists_euc.append(intra_hist_euc[0])
            inter_hists_euc.append(inter_hist_euc[0])
            intra_hists_man.append(intra_hist_man[0])
            inter_hists_man.append(inter_hist_man[0])
            intra_hists_cos.append(intra_hist_cos[0])
            inter_hists_cos.append(inter_hist_cos[0])
            
            ratios_euc.append(ratio_euc)
            ratios_man.append(ratio_man)
            ratios_cos.append(ratio_cos)
            ratios_dsne.append(ratio_dsne)
            ratios_dsne2.append(ratio_dsne2)
            ratios_dsne5.append(ratio_dsne5)
            ratios_dsne10.append(ratio_dsne10)

            means_euc.append(mean_euc)
            means_man.append(mean_man)
            means_cos.append(mean_cos)
            means_dsne.append(mean_dsne)
            means_dsne2.append(mean_dsne2)
            means_dsne5.append(mean_dsne5)
            means_dsne10.append(mean_dsne10)

            stdevs_euc.append(stdev_euc)
            stdevs_man.append(stdev_man)
            stdevs_cos.append(stdev_cos)
            stdevs_dsne.append(stdev_dsne)
            stdevs_dsne2.append(stdev_dsne2)
            stdevs_dsne5.append(stdev_dsne5)
            stdevs_dsne10.append(stdev_dsne10)

        bin_man = intra_hist_man[1]
        bin_euc = intra_hist_euc[1]
        bin_cos = intra_hist_cos[1]

        bincenters = np.array(range(num_bin)) + 0.5
        
        
        dsnebins = np.arange(0,(num_bin)*8,8)
        dsnebins2 = np.arange(1,(num_bin)*8,8)
        dsnebins5 = np.arange(2,(num_bin)*8,8)
        dsnebins10 = np.arange(3,(num_bin)*8,8)
        eucbins = np.arange(4,(num_bin)*8,8)
        manbins = np.arange(5,(num_bin)*8,8)
        cosbins = np.arange(6,(num_bin)*8,8)
        
        maxdsnebin = np.max(np.add(inter_hist_dsne[0],intra_hist_dsne[0])/2)
        maxdsne2bin = np.max(np.add(inter_hist_dsne2[0],intra_hist_dsne2[0])/2)
        maxdsne5bin = np.max(np.add(inter_hist_dsne5[0],intra_hist_dsne5[0])/2)
        maxdsne10bin = np.max(np.add(inter_hist_dsne10[0],intra_hist_dsne10[0])/2)
        maxeucbin = np.max(np.add(inter_hist_euc[0],intra_hist_euc[0])/2)
        maxmanbin = np.max(np.add(inter_hist_man[0],intra_hist_man[0])/2)
        maxcosbin = np.max(np.add(inter_hist_cos[0],intra_hist_cos[0])/2)
        
        maxbin = max(maxdsnebin, maxdsne2bin)
        maxbin = max(maxbin, maxdsne5bin)
        maxbin = max(maxbin, maxdsne10bin)
        maxbin = max(maxbin, maxeucbin)
        maxbin = max(maxbin, maxmanbin)
        maxbin = max(maxbin, maxcosbin)
        
        plt.figure(figsize=(14,4))

        plt.title(title + " Dimensionality: " + str(dimlist[i]))
        
        
        #print(intra_hists_dsne)
        
        for j in range(len(dsnebins)):
            inter_hist_dsnei = 0
            intra_hist_dsnei = 0
            
            for s in range(len(seedlist)):
                inter_hist_dsnei = inter_hist_dsnei + inter_hists_dsne[s][j]
                intra_hist_dsnei = intra_hist_dsnei + intra_hists_dsne[s][j]
                
            inter_hist_dsnei = inter_hist_dsnei/len(seedlist)
            intra_hist_dsnei = intra_hist_dsnei/len(seedlist)
            
            if(inter_hist_dsnei > intra_hist_dsnei):
                plt.bar(dsnebins[j], inter_hist_dsnei/2, label="Inter-Class DSNE", color=colors[1], edgecolor='black', width=0.6)        
                plt.bar(dsnebins[j]-0.2, intra_hist_dsnei/2, label="Intra-Class DSNE", color=colors[0], edgecolor='black', width=0.6)
            else:
                plt.bar(dsnebins[j]-0.2, intra_hist_dsnei/2, label="Intra-Class DSNE", color=colors[0], edgecolor='black', width=0.6)
                plt.bar(dsnebins[j], inter_hist_dsnei/2, label="Inter-Class DSNE", color=colors[1], edgecolor='black', width=0.6)        
            
            inter_hist_dsne2i = 0
            intra_hist_dsne2i = 0
            
            for s in range(len(seedlist)):
                inter_hist_dsne2i = inter_hist_dsne2i + inter_hists_dsne2[s][j]
                intra_hist_dsne2i = intra_hist_dsne2i + intra_hists_dsne2[s][j]
                
            inter_hist_dsne2i = inter_hist_dsne2i/len(seedlist)
            intra_hist_dsne2i = intra_hist_dsne2i/len(seedlist)
            
            if(inter_hist_dsne2i > intra_hist_dsne2i):
                plt.bar(dsnebins2[j], inter_hist_dsne2i/2, label="Inter-Class DSNE mP=2", color=colors[3], edgecolor='black', width=0.6, hatch="///")        
                plt.bar(dsnebins2[j]-0.2, intra_hist_dsne2i/2, label="Intra-Class DSNE mP=2", color=colors[2], edgecolor='black', width=0.6, hatch="///")
            else:
                plt.bar(dsnebins2[j]-0.2, intra_hist_dsne2i/2, label="Intra-Class DSNE mP=2", color=colors[2], edgecolor='black', width=0.6, hatch="///")
                plt.bar(dsnebins2[j], inter_hist_dsne2i/2, label="Inter-Class DSNE mP=2", color=colors[3], edgecolor='black', width=0.6, hatch="///")        

            inter_hist_dsne5i = 0
            intra_hist_dsne5i = 0
            
            for s in range(len(seedlist)):
                inter_hist_dsne5i = inter_hist_dsne5i + inter_hists_dsne5[s][j]
                intra_hist_dsne5i = intra_hist_dsne5i + intra_hists_dsne5[s][j]
                
            inter_hist_dsne5i = inter_hist_dsne5i/len(seedlist)
            intra_hist_dsne5i = intra_hist_dsne5i/len(seedlist)
            
            if(inter_hist_dsne5i > intra_hist_dsne5i):
                plt.bar(dsnebins5[j], inter_hist_dsne5i/2, label="Inter-Class DSNE mP=5", color=colors[5], edgecolor='black', width=0.6, hatch='\\\\\\')        
                plt.bar(dsnebins5[j]-0.2, intra_hist_dsne5i/2, label="Intra-Class DSNE mP=5", color=colors[4], edgecolor='black', width=0.6, hatch='\\\\\\')
            else:
                plt.bar(dsnebins5[j]-0.2, intra_hist_dsne5i/2, label="Intra-Class DSNE mP=5", color=colors[4], edgecolor='black', width=0.6, hatch='\\\\\\')
                plt.bar(dsnebins5[j], inter_hist_dsne5i/2, label="Inter-Class DSNE mP=5", color=colors[5], edgecolor='black', width=0.6, hatch='\\\\\\')        

                
            inter_hist_dsne10i = 0
            intra_hist_dsne10i = 0
            
            for s in range(len(seedlist)):
                inter_hist_dsne10i = inter_hist_dsne10i + inter_hists_dsne10[s][j]
                intra_hist_dsne10i = intra_hist_dsne10i + intra_hists_dsne10[s][j]
                
            inter_hist_dsne10i = inter_hist_dsne10i/len(seedlist)
            intra_hist_dsne10i = intra_hist_dsne10i/len(seedlist)
            
            if(inter_hist_dsne10i > intra_hist_dsne10i):
                plt.bar(dsnebins10[j], inter_hist_dsne10i/2, label="Inter-Class DSNE mP=10", color=colors[7], edgecolor='black', width=0.6, hatch="xxx")        
                plt.bar(dsnebins10[j]-0.2, intra_hist_dsne10i/2, label="Intra-Class DSNE mP=10", color=colors[6], edgecolor='black', width=0.6, hatch="xxx")
            else:
                plt.bar(dsnebins10[j]-0.2, intra_hist_dsne10i/2, label="Intra-Class DSNE mP=10", color=colors[6], edgecolor='black', width=0.6, hatch="xxx")
                plt.bar(dsnebins10[j], inter_hist_dsne10i/2, label="Inter-Class DSNE mP=10", color=colors[7], edgecolor='black', width=0.6, hatch="xxx")        

            inter_hist_euci = 0
            intra_hist_euci = 0
            
            for s in range(len(seedlist)):
                inter_hist_euci = inter_hist_euci + inter_hists_euc[s][j]
                intra_hist_euci = intra_hist_euci + intra_hists_euc[s][j]
                
            inter_hist_euci = inter_hist_euci/len(seedlist)
            intra_hist_euci = intra_hist_euci/len(seedlist)
            
            #print(inter_hist_euci)
            
            if(inter_hist_euci > intra_hist_euci):
                plt.bar(eucbins[j], inter_hist_euci/2, label="Inter-Class Euclidean", color=colors[9], edgecolor='black', width=0.6)        
                plt.bar(eucbins[j]-0.2, intra_hist_euci/2, label="Intra-Class Euclidean", color=colors[8], edgecolor='black', width=0.6)
            else:
                plt.bar(eucbins[j]-0.2, intra_hist_euci/2, label="Intra-Class Euclidean", color=colors[8], edgecolor='black', width=0.6)
                plt.bar(eucbins[j], inter_hist_euci/2, label="Inter-Class Euclidean", color=colors[9], edgecolor='black', width=0.6)      
                
            inter_hist_mani = 0
            intra_hist_mani = 0
            
            for s in range(len(seedlist)):
                inter_hist_mani = inter_hist_mani + inter_hists_man[s][j]
                intra_hist_mani = intra_hist_mani + intra_hists_man[s][j]
                
            inter_hist_mani = inter_hist_mani/len(seedlist)
            intra_hist_mani = intra_hist_mani/len(seedlist)
            
            if(inter_hist_mani > intra_hist_mani):
                plt.bar(manbins[j], inter_hist_mani/2, label="Inter-Class Manhattan", color=colors[11], edgecolor='black', width=0.6)        
                plt.bar(manbins[j]-0.2, intra_hist_mani/2, label="Intra-Class Manhattan", color=colors[10], edgecolor='black', width=0.6)
            else:
                plt.bar(manbins[j]-0.2, intra_hist_mani/2, label="Intra-Class Manhattan", color=colors[10], edgecolor='black', width=0.6)
                plt.bar(manbins[j], inter_hist_mani/2, label="Inter-Class Manhattan", color=colors[11], edgecolor='black', width=0.6)        
            
            inter_hist_cosi = 0
            intra_hist_cosi = 0
            
            for s in range(len(seedlist)):
                inter_hist_cosi = inter_hist_cosi + inter_hists_cos[s][j]
                intra_hist_cosi = intra_hist_cosi + intra_hists_cos[s][j]
                
            inter_hist_cosi = inter_hist_cosi/len(seedlist)
            intra_hist_cosi = intra_hist_cosi/len(seedlist)
            
            if(inter_hist_cosi > intra_hist_cosi):
                plt.bar(cosbins[j], inter_hist_cosi/2, label="Inter-Class Cosine", color=colors[13], edgecolor='black', width=0.6)        
                plt.bar(cosbins[j]-0.2, intra_hist_cosi/2, label="Intra-Class Cosine", color=colors[12], edgecolor='black', width=0.6)
            else:
                plt.bar(cosbins[j]-0.2, intra_hist_cosi/2, label="Intra-Class Cosine", color=colors[12], edgecolor='black', width=0.6)
                plt.bar(cosbins[j], inter_hist_cosi/2, label="Inter-Class Cosine", color=colors[13], edgecolor='black', width=0.6)        
                
        plt.vlines(cosbins + 1, 0, 1.1*maxbin, color='black', linestyles='--')
        plt.xticks(dsnebins10,(np.arange(num_bin)+1))
        plt.xlim(-0.9, np.max(cosbins)+ 0.9)
        
        plt.ylabel('number of occurences')
        plt.xlabel('10% of maximal distance')
        #plt.legend(framealpha = 1)
        plt.savefig(str(datasets[k]) + "_dists.pdf")


        plt.show()
        
            
    plt.figure(figsize=(14,4))
    plt.title(title + " Min-Max Ratio")
    
    
    ratios_dsne_np = np.array(ratios_dsne).reshape(len(dimlist),len(seedlist))
    ratios_dsne2_np = np.array(ratios_dsne2).reshape(len(dimlist),len(seedlist))
    ratios_dsne5_np = np.array(ratios_dsne5).reshape(len(dimlist),len(seedlist))
    ratios_dsne10_np = np.array(ratios_dsne10).reshape(len(dimlist),len(seedlist))

    ratios_euc_np = np.array(ratios_euc).reshape(len(dimlist),len(seedlist))
    ratios_man_np = np.array(ratios_man).reshape(len(dimlist),len(seedlist))
    ratios_cos_np = np.array(ratios_cos).reshape(len(dimlist),len(seedlist))

    
    ratiosmean_dsne = np.mean(ratios_dsne_np,1)
    ratiosstdev_dsne = np.std(ratios_dsne_np,1)
    
    ratiosmean_dsne2 = np.mean(ratios_dsne2_np,1)
    ratiosstdev_dsne2 = np.std(ratios_dsne2_np,1)
    
    ratiosmean_dsne5 = np.mean(ratios_dsne5_np,1)
    ratiosstdev_dsne5 = np.std(ratios_dsne5_np,1)
    
    ratiosmean_dsne10 = np.mean(ratios_dsne10_np,1)
    ratiosstdev_dsne10 = np.std(ratios_dsne10_np,1)
    
    ratiosmean_euc = np.mean(ratios_euc_np,1)
    ratiosstdev_euc = np.std(ratios_euc_np,1)
    
    ratiosmean_man = np.mean(ratios_man_np,1)
    ratiosstdev_man = np.std(ratios_man_np,1)
    
    ratiosmean_cos = np.mean(ratios_cos_np,1)
    ratiosstdev_cos = np.std(ratios_cos_np,1)
    
    #print(ratiosmean_dsne)
    
    colors = plt.cm.tab20(np.linspace(0, 1, 20))

    
    plt.plot(dims, ratiosmean_dsne, label="DSNE", color=colors[0])
    plt.plot(dims, ratiosmean_dsne2, label="DSNE mP=2", color=colors[2])
    plt.plot(dims, ratiosmean_dsne5, label="DSNE mP=5", color=colors[4])
    plt.plot(dims, ratiosmean_dsne10, label="DSNE mP=10", color=colors[6])
    plt.plot(dims, ratiosmean_euc, label="Euclidean", color=colors[8])
    plt.plot(dims, ratiosmean_man, label="Manhattan", color=colors[10])
    plt.plot(dims, ratiosmean_cos, label="Cosine", color=colors[12])
    
    plt.plot(dims, ratiosmean_dsne + ratiosstdev_dsne, color=colors[1], linestyle='--' )
    plt.plot(dims, ratiosmean_dsne - ratiosstdev_dsne, color=colors[1], linestyle='--' )
    
    plt.plot(dims, ratiosmean_dsne2 + ratiosstdev_dsne2, color=colors[3], linestyle='--' )
    plt.plot(dims, ratiosmean_dsne2 - ratiosstdev_dsne2, color=colors[3], linestyle='--' )
    
    plt.plot(dims, ratiosmean_dsne5 + ratiosstdev_dsne5, color=colors[5], linestyle='--' )
    plt.plot(dims, ratiosmean_dsne5 - ratiosstdev_dsne5, color=colors[5], linestyle='--' )
    
    plt.plot(dims, ratiosmean_dsne10 + ratiosstdev_dsne10, color=colors[7], linestyle='--' )
    plt.plot(dims, ratiosmean_dsne10 - ratiosstdev_dsne10, color=colors[7], linestyle='--' )
    
    plt.plot(dims, ratiosmean_euc + ratiosstdev_euc, color=colors[9], linestyle='--' )
    plt.plot(dims, ratiosmean_euc - ratiosstdev_euc, color=colors[9], linestyle='--' )
    
    plt.plot(dims, ratiosmean_man + ratiosstdev_man, color=colors[11], linestyle='--' )
    plt.plot(dims, ratiosmean_man - ratiosstdev_man, color=colors[11], linestyle='--' )
    
    plt.plot(dims, ratiosmean_cos + ratiosstdev_cos, color=colors[13], linestyle='--' )
    plt.plot(dims, ratiosmean_cos - ratiosstdev_cos, color=colors[13], linestyle='--' )   

    
    plt.ylabel('minimal distance / maximal distance')
    plt.xlabel('dimensionality')
    
    

    #plt.legend()
    plt.savefig(title.lower() + "_minmaxratio.pdf")

    plt.show()

In [ ]:
print(datasets)
num_bin = 10
make_graph(points_list, labels_list, num_bin, eucliddists, manhatdists, cosinedists, dsnedists, dsnedists2, dsnedists5, dsnedists10, dims, seeds, datasets)#, dsnedist2)